Set Animal IDs and Experiment IDs

In [1]:
data_path = '//Users//ryan.gorzek//Downloads//Mapping_Through_Skull//'

exp_info = {
            'DR_ctb01': {'exp_name': '20220603_101509', 'rearing': 'DR'},
            'DR_ctb02': {'exp_name': '20220610_105811', 'rearing': 'DR'},
            'DR_ctb07': {'exp_name': '20220711_135556', 'rearing': 'DR'},
            'DR_ctb08': {'exp_name': '20220715_121356', 'rearing': 'DR'},
            'DR_lt168': {'exp_name': '20220330_130327', 'rearing': 'DR'},
            'DR_lt169': {'exp_name': '20220420_125809', 'rearing': 'DR'},
            'DR_lt170': {'exp_name': '20220421_133932', 'rearing': 'DR'},
            'NR_ctb03': {'exp_name': '20220623_161247', 'rearing': 'NR'},
            'NR_ctb04': {'exp_name': '20220630_130119', 'rearing': 'NR'},
            'NR_ctb05': {'exp_name': '20220706_105937', 'rearing': 'NR'},
            'NR_ctb09': {'exp_name': '20220720_153020', 'rearing': 'NR'},
            'NR_lt164': {'exp_name': '20220302_131952', 'rearing': 'NR'},
            'NR_lt171': {'exp_name': '20220427_115917', 'rearing': 'NR'},
            'NR_lt172': {'exp_name': '20220428_130342', 'rearing': 'NR'},
            'NR_lt175': {'exp_name': '20220512_124014', 'rearing': 'NR'}
            }

area_list = ['V1', 'LM', 'AL', 'RL', 'AM', 'PM', 'M']

Get User Input for Visual Area Boundaries on Sign Maps

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
from os.path import exists

%matplotlib qt5
for exp in exp_info:
    curr_path = data_path + '{a}//{e}//'.format(a=exp, e=exp_info[exp]['exp_name'])
    exp_info[exp]['area_coords'] = dict(zip(area_list, [None]*len(area_list)))
    refframe = imread(curr_path + 'refframe.tif').squeeze()
    signmap = np.loadtxt(curr_path + 'signmap.csv').squeeze()
    for area_num,area in enumerate(area_list):
        fig, ax = plt.subplots()
        ax.imshow(refframe, cmap='gray')
        ax.imshow(signmap, cmap='RdBu_r', clim=[-1,1], alpha=0.3)
        if area_num > 0:
            for prev_area in area_list[:area_num]:
                ax.plot(exp_info[exp]['area_coords'][prev_area][:,0], exp_info[exp]['area_coords'][prev_area][:,1], color='k', linewidth=3)
        ax.set_title('Select Region for {ar}:'.format(ar=area), fontsize=20)
        ax.axis('off')
        fig.set_size_inches(12, 12)
        if exists(curr_path + '{ar}.csv'.format(ar=area)):
            exp_info[exp]['area_coords'][area] = np.loadtxt(curr_path + '{ar}.csv'.format(ar=area))
        else:
            curr_input = plt.ginput(n=-1, timeout=-1)
            curr_input = np.array(curr_input)
            curr_input = np.append(curr_input, [curr_input[0,:]], axis=0)
            exp_info[exp]['area_coords'][area] = curr_input
            np.savetxt(curr_path + '{ar}.csv'.format(ar=area), curr_input)
        plt.close()